In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, feature_selection, linear_model, cross_validation, decomposition, cluster
import matplotlib.pyplot as plt
import statsmodels.api as sm


C:\Users\bruce\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\bruce\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df1 = pd.DataFrame.from_csv('stock_data/factor_data.csv',index_col=10)
df2 = pd.DataFrame.from_csv('stock_data/ticker_data.csv',index_col=['timestep','ticker'])
df2 = df2.unstack()
df2 = df2.fillna(0)
clean_upcols = {'industry':{'TECH':0,'AGRICULTURE':1,'FINANCE':2,'CONSUMER':3,'OTHER':4}}
df2.replace(clean_upcols,inplace=True)
df2 = df2.drop(['index'],axis=1)

C:\Users\bruce\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
C:\Users\bruce\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


In [3]:
x=pd.DataFrame()
x=df1[['3M_R','BIG_IX','SMALL_IX']]
big = df2['returns'].where(df2['market_cap'].rank(1)>800,0)
big = big.sum(1)/200
low = df2['returns'].where(df2['pb'].rank(1)>800,0)
low = low.sum(1)/200
small = df2['returns'].where(df2['market_cap'].rank(1)<200,0)
small = small.sum(1)/200
high = df2['returns'].where(df2['pb'].rank(1)<200,0)
high = high.sum(1)/200
market=df2['returns'].sum(1)/1000 - df1['3M_R']
x['BIG']=big
x['SMALL']=small
x['market']= market
x['high']=high
x['low']=low
x=x.drop(['BIG_IX','SMALL_IX'],axis=1)
#add constant adds an array of ones, its how statsmodel gives you the beta0/alpha
x = sm.add_constant(x)

C:\Users\bruce\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\bruce\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\bruce\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

In [4]:
#Change 0 in returns to whatever ticker, run on market if you want market betas
y = df2['returns',0] - df1['3M_R']

In [5]:
model = sm.OLS(endog=y,exog=x)
res = model.fit()
res.rsquared_adj

0.9721644680159929

In [98]:
alphas=[]
for i in range(1000):
    y = df2['returns',i] - df1['3M_R']
    model = sm.OLS(endog=y,exog=x)
    res = model.fit()
    if res.rsquared_adj<0.9:
        print('unmatched')
        break
    else:
        alphas.append(res.params[0])
df_a=pd.DataFrame(alphas)

In [51]:
(df_a.quantile([0.90]).iloc[0]+df_a.quantile([0.95]).iloc[0])/2
#((df_a.quantile([0.95]).iloc[0]-df_a.quantile([0.90]).iloc[0]))

0    0.000468
dtype: float64

In [9]:
first=df2['returns'].iloc[49]

In [10]:
first.max()

0.011988655908924615

In [30]:
abs(df2['returns'].mean()).std()

4.512024165482914e-05

In [31]:
abs(df_a).std()

0    0.000185
dtype: float64

In [ ]:
a=df_a.quantile([0.90]).iloc[0][0]
b=df_a.quantile([0.95]).iloc[0][0]

In [ ]:
a

In [ ]:
b

In [ ]:
interval=first[(first >= a) & (first<=b)]
interval

In [ ]:
interval[50]-interval.median()

In [34]:
def move_lower(a,b,l):
    if a>b:
        print('a should be less than b')
        return -1
    else:
        return(a-l,b-l)

In [ ]:
1e-08

In [89]:
def weighted_median(value,median,gap):
    my_weight=1
    if abs(value-median)<=1e-08:
        my_weight=gap/abs(value-median+1e-08)
    else:
        my_weight=gap/abs(value-median)
    return my_weight

In [ ]:
weights=interval.apply(weighted_median,args=(interval.median(),(interval.max()-interval.min())))

In [128]:
def portfolio_weights(pred_returns,df_a):
    a=0.9
    b=0.95
    lower=df_a.quantile([a]).iloc[0][0]
    upper=df_a.quantile([b]).iloc[0][0]
    target_interval=pred_returns[(pred_returns >= lower) & (pred_returns<=upper)]
    while len(target_interval)==0:
        a,b=move_lower(a,b,0.05)
        lower=df_a.quantile([a]).iloc[0][0]
        upper=df_a.quantile([b]).iloc[0][0]
        target_interval=pred_returns[(pred_returns >= lower) & (pred_returns<=upper)]

    my_weights=np.zeros(1000)
    temp_weight=target_interval.apply(weighted_median,args=((lower+upper)/2,(target_interval.max()-target_interval.min())))
    for i in temp_weight.index.values:
        #print(i)
        my_weights[i]=temp_weight[i]
        """
        for i in target_interval.index.values:
        my_weights[i]=1"""
    return my_weights/my_weights.sum()

In [13]:
from math import sqrt
from sklearn import preprocessing, feature_selection, linear_model, cross_validation, decomposition, cluster

In [63]:
#my_list=[]
for i in range(50,2520):
    #weights=portfolio_weights(df2['returns'].iloc[i],df_a)
    ea=df2['returns'].iloc[i][:500].mean()
    eb=df2['returns'].iloc[i][500:].mean()
    lambda1=(1e-04-eb)/(ea-eb)
    weights=np.ones(1000)
    weights[:500]*=lambda1/500
    weights[500:]*=(1-lambda1)/500
    value=np.dot(weights,df2['returns'].iloc[i])
    #my_list.append(value)
    #print(lambda1)

In [138]:
pd.DataFrame(my_list).to_csv('l1.csv')

In [177]:
c=pd.DataFrame(my_list)

In [180]:
sqrt(252)*c.mean()/c.std()

0    3.640075e+14
dtype: float64

In [5]:
weights = pd.read_csv('weights.csv')
feature_coefs = pd.read_csv('feature_coefs.csv')

In [8]:
weights=weights.drop('Unnamed: 0',axis=1)

In [16]:
poly = preprocessing.PolynomialFeatures(4)
transformed_features=pd.DataFrame(poly.fit_transform(df1[['VIX',	'COPP',	'3M_R', 'US_TRY','BIG_IX',	'SMALL_IX', 'SENTI','TEMP',	'RAIN',	'OIL']].iloc[49].reshape(1,-1)))

C:\Users\bruce\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [17]:
transformed_features = transformed_features.fillna(0)
big = (np.transpose(feature_coefs['BIG_IX'])*transformed_features).sum()
small = (np.transpose(feature_coefs['SMALL_IX'])*(transformed_features)).sum()
risk_free = (np.transpose(feature_coefs['3M_R'])*(transformed_features)).sum()
pred_returns=weights['3M_R']*risk_free.sum() + weights['BIG_IX']*big.sum()+weights['SMALL_IX']* small.sum() + risk_free.sum()

In [22]:
pred_returns[:500].sum()-df2['returns'].iloc[50][:500].sum()

-0.0723325597941952

In [23]:
pred_returns[:500].sum()

-0.0033699415998416377

In [45]:
abs(df2['returns'].loc[:,:499].mean(1)).mean()

0.0001134822465987188

In [46]:
abs(df2['returns'].loc[:,500:].mean(1)).mean()

0.00011969277894009956

In [49]:
import random

In [52]:
len(set(random.sample(range(1000),500)))

500

In [53]:
a=random.sample(range(1000),500)

In [62]:
len([i for i in range(1000) if i not in a])

500

In [76]:
df2['returns'].loc[:,0:9].min().min()

-0.01188198462115675

In [5]:
MAX_LOOKBACK = 50

class PortfolioGenerator(object):

    def __init__(self):
        pass

    def read_stock_data(self):
        '''
        Description:
            Reads in simulated stock data from stock_data.csv
        Returns:
            stock_df (DataFrame): standardized ticker/factor data in pandas df
        Raises:
            AssertionError: ticker_data.csv/factor_data.csv has an invalid format
        '''
        ticker_df = pd.read_csv('stock_data/ticker_data.csv')
        factor_df = pd.read_csv('stock_data/factor_data.csv')
        assert 'timestep' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'ticker' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'returns' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'timestep' in factor_df.columns, "factor_data.csv has an invalid format"
        ticker_df.set_index('timestep', inplace=True)
        factor_df.set_index('timestep', inplace=True)
        stock_df = ticker_df.join(factor_df, how='left')
        return stock_df

    def build_signal(self, stock_features):
        '''
        Description:
            Using stock features, generate a buy/sell stock
            signal (suggested from -100 to 100 where -100 is strong
            sell signal) for the stock on a given day.
        Args:
            hist_features (DataFrame) - dataframe of historical features
        Return:
            signal (pandas Series) - key = ticker, value = signal
        Raises:
            NotImplementedError - throws error if the function is not implemented
        '''
        raise NotImplementedError("build_signal must be implemented")

    def simulate_portfolio(self):
        '''
        Description:
            Simulates performance of the portfolio on historical data
        Return:
            sharpe (int) - sharpe ratio for the portfolio
        '''
        daily_returns = []
        stock_df = self.read_stock_data()
        for idx in stock_df.index.unique():
            print("timestep", idx)
            if idx < MAX_LOOKBACK:
                continue
            stock_features = stock_df.loc[idx-MAX_LOOKBACK:idx-1]
            returns = stock_df.loc[idx:idx].set_index('ticker')['returns']
            signal = self.build_signal(stock_features)
            signal_return = returns *signal
            daily_returns.append(np.mean(signal_return))
            #print(daily_returns[idx-50])
        sharpe_ratio = np.sqrt(252) * (np.mean(daily_returns) / np.std(daily_returns))
        #pd.DataFrame(daily_returns,columns=['a']).to_csv('dailyreturn_market.csv')
        
        return sharpe_ratio

In [6]:
class SampleStrategy(PortfolioGenerator):

    def __init__(self):
        pass

    def build_signal(self, stock_features):
        return self.momentum(stock_features)

    def momentum(self, stock_features):
        i=stock_features['returns'].index.values[-1]+1
        #print(list(stock_features[['VIX','COPP','3M_R','US_TRY','BIG_IX','SMALL_IX','SENTI','TEMP','RAIN','OIL']].iloc[-1]))
        ea=df2['returns'].iloc[i][:500].mean()
        eb=df2['returns'].iloc[i][500:].mean()
        lambda1=(1e-04-eb)/(ea-eb)
        my_weights=np.ones(1000)
        my_weights[:500]*=lambda1/500
        my_weights[500:]*=(1-lambda1)/500

        #print(pred_returns[0])
        return my_weights
# Test out performance by running 'python sample_strategy.py'
if __name__ == "__main__":
    portfolio = SampleStrategy()
    sharpe = portfolio.simulate_portfolio()
    print("*** Strategy Sharpe is {} ***".format(sharpe))

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 5
timestep 6
timestep 7
timestep 8
timestep 9
timestep 10
timestep 11
timestep 12
timestep 13
timestep 14
timestep 15
timestep 16
timestep 17
timestep 18
timestep 19
timestep 20
timestep 21
timestep 22
timestep 23
timestep 24
timestep 25
timestep 26
timestep 27
timestep 28
timestep 29
timestep 30
timestep 31
timestep 32
timestep 33
timestep 34
timestep 35
timestep 36
timestep 37
timestep 38
timestep 39
timestep 40
timestep 41
timestep 42
timestep 43
timestep 44
timestep 45
timestep 46
timestep 47
timestep 48
timestep 49
timestep 50
timestep 51
timestep 52
timestep 53
timestep 54
timestep 55
timestep 56
timestep 57
timestep 58
timestep 59
timestep 60
timestep 61
timestep 62
timestep 63
timestep 64
timestep 65
timestep 66
timestep 67
timestep 68
timestep 69
timestep 70
timestep 71
timestep 72
timestep 73
timestep 74
timestep 75
timestep 76
timestep 77
timestep 78
timestep 79
timestep 80
timestep 81
timestep 82
timestep 83
ti

timestep 643
timestep 644
timestep 645
timestep 646
timestep 647
timestep 648
timestep 649
timestep 650
timestep 651
timestep 652
timestep 653
timestep 654
timestep 655
timestep 656
timestep 657
timestep 658
timestep 659
timestep 660
timestep 661
timestep 662
timestep 663
timestep 664
timestep 665
timestep 666
timestep 667
timestep 668
timestep 669
timestep 670
timestep 671
timestep 672
timestep 673
timestep 674
timestep 675
timestep 676
timestep 677
timestep 678
timestep 679
timestep 680
timestep 681
timestep 682
timestep 683
timestep 684
timestep 685
timestep 686
timestep 687
timestep 688
timestep 689
timestep 690
timestep 691
timestep 692
timestep 693
timestep 694
timestep 695
timestep 696
timestep 697
timestep 698
timestep 699
timestep 700
timestep 701
timestep 702
timestep 703
timestep 704
timestep 705
timestep 706
timestep 707
timestep 708
timestep 709
timestep 710
timestep 711
timestep 712
timestep 713
timestep 714
timestep 715
timestep 716
timestep 717
timestep 718
timestep 719

timestep 1258
timestep 1259
timestep 1260
timestep 1261
timestep 1262
timestep 1263
timestep 1264
timestep 1265
timestep 1266
timestep 1267
timestep 1268
timestep 1269
timestep 1270
timestep 1271
timestep 1272
timestep 1273
timestep 1274
timestep 1275
timestep 1276
timestep 1277
timestep 1278
timestep 1279
timestep 1280
timestep 1281
timestep 1282
timestep 1283
timestep 1284
timestep 1285
timestep 1286
timestep 1287
timestep 1288
timestep 1289
timestep 1290
timestep 1291
timestep 1292
timestep 1293
timestep 1294
timestep 1295
timestep 1296
timestep 1297
timestep 1298
timestep 1299
timestep 1300
timestep 1301
timestep 1302
timestep 1303
timestep 1304
timestep 1305
timestep 1306
timestep 1307
timestep 1308
timestep 1309
timestep 1310
timestep 1311
timestep 1312
timestep 1313
timestep 1314
timestep 1315
timestep 1316
timestep 1317
timestep 1318
timestep 1319
timestep 1320
timestep 1321
timestep 1322
timestep 1323
timestep 1324
timestep 1325
timestep 1326
timestep 1327
timestep 1328
timest

timestep 1845
timestep 1846
timestep 1847
timestep 1848
timestep 1849
timestep 1850
timestep 1851
timestep 1852
timestep 1853
timestep 1854
timestep 1855
timestep 1856
timestep 1857
timestep 1858
timestep 1859
timestep 1860
timestep 1861
timestep 1862
timestep 1863
timestep 1864
timestep 1865
timestep 1866
timestep 1867
timestep 1868
timestep 1869
timestep 1870
timestep 1871
timestep 1872
timestep 1873
timestep 1874
timestep 1875
timestep 1876
timestep 1877
timestep 1878
timestep 1879
timestep 1880
timestep 1881
timestep 1882
timestep 1883
timestep 1884
timestep 1885
timestep 1886
timestep 1887
timestep 1888
timestep 1889
timestep 1890
timestep 1891
timestep 1892
timestep 1893
timestep 1894
timestep 1895
timestep 1896
timestep 1897
timestep 1898
timestep 1899
timestep 1900
timestep 1901
timestep 1902
timestep 1903
timestep 1904
timestep 1905
timestep 1906
timestep 1907
timestep 1908
timestep 1909
timestep 1910
timestep 1911
timestep 1912
timestep 1913
timestep 1914
timestep 1915
timest

timestep 2436
timestep 2437
timestep 2438
timestep 2439
timestep 2440
timestep 2441
timestep 2442
timestep 2443
timestep 2444
timestep 2445
timestep 2446
timestep 2447
timestep 2448
timestep 2449
timestep 2450
timestep 2451
timestep 2452
timestep 2453
timestep 2454
timestep 2455
timestep 2456
timestep 2457
timestep 2458
timestep 2459
timestep 2460
timestep 2461
timestep 2462
timestep 2463
timestep 2464
timestep 2465
timestep 2466
timestep 2467
timestep 2468
timestep 2469
timestep 2470
timestep 2471
timestep 2472
timestep 2473
timestep 2474
timestep 2475
timestep 2476
timestep 2477
timestep 2478
timestep 2479
timestep 2480
timestep 2481
timestep 2482
timestep 2483
timestep 2484
timestep 2485
timestep 2486
timestep 2487
timestep 2488
timestep 2489
timestep 2490
timestep 2491
timestep 2492
timestep 2493
timestep 2494
timestep 2495
timestep 2496
timestep 2497
timestep 2498
timestep 2499
timestep 2500
timestep 2501
timestep 2502
timestep 2503
timestep 2504
timestep 2505
timestep 2506
timest